<b>Note: This Jupyter Notebook is associated with the article [How To Trade 0DTE Options with Alpaca's Trading API (in Python)](https://alpaca.markets/learn/how-to-trade-0dte-options-on-alpaca).</b>

# Step 1: Setting Up the Environment and Trade Parameters
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
from scipy.optimize import brentq
from datetime import datetime, time
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ClosePositionRequest,
)
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import OptionLatestQuoteRequest
from alpaca.trading.enums import (
    AssetStatus,
    # ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderType,
    TimeInForce,
    # QueryOrderStatus,
    ContractType
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for developing this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

# Step 2: Choosing the Right Strike Price, Expiration, and Other Key Factors

In [ ]:
# Configuration
underlying_symbol = 'SPY'

# Set the timezone
timezone = ZoneInfo("America/New_York")

# Set expiration range for options
today = datetime.now(timezone).date()

# Define a 10% range around the underlying price
STRIKE_RANGE = 0.1

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Set the open interest volume threshold
OI_THRESHOLD = 500

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.10

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Define delta thresholds
SHORT_PUT_DELTA_RANGE = (-0.42, -0.38)
LONG_PUT_DELTA_RANGE = (-0.22, -0.18)

# Set minimum credit percentage (33%)
MIN_CREDIT_PERCENTAGE = 0.33

# Set stop loss threshold threshold (2 times)
DELTA_STOP_LOSS_THRES = 2

# Set target profit and stop-loss levels
TARGET_STOP_LOSS_PERCENTAGE = 0.5

In [ ]:
# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)
print(f"{underlying_symbol} price: {underlying_price}")

# Step3: Building a Function to Extract a List of 0DTE options

In [ ]:
# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Function to check for 0DTE options
def get_0DTE_options(underlying_symbol):
    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(underlying_symbols=[underlying_symbol],
                                    strike_price_gte=min_strike,
                                    strike_price_lte=max_strike,
                                    status=AssetStatus.ACTIVE,
                                    expiration_date=today,
                                    root_symbol=underlying_symbol,
                                    type=ContractType.PUT,
                                    )

    # Get option chain (zero_dte_option) of the underlying symbol
    zero_dte_options = trade_client.get_option_contracts(req).option_contracts

    return zero_dte_options

# Step4: Building a Function to Calculate Implied Volatility and Option Greek (Delta)

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):

    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:

        # print("Option price is close to intrinsic value; implied volatility is near zero.") # Uncomment for checking the status

        return 0.0

    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None


# Calculate option Delta
def calculate_delta(option_price, strike_price, expiry, underlying_price, risk_free_rate, option_type):
    # Set the current date and time
    now = datetime.now(tz=timezone)
    # Calculate the time to expiry in years
    T = (expiry - now).total_seconds() / (365 * 24 * 60 * 60)
    # Set minimum T to avoid zero
    T = max(T, 1e-6)

    if T == 1e-6:
        print("Option has expired or is expiring now; setting delta based on intrinsic value.")
        if option_type == 'put':
            return -1.0 if underlying_price < strike_price else 0.0
        else:
            return 1.0 if underlying_price > strike_price else 0.0

    implied_volatility = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)
    if implied_volatility is None:
        print("Implied volatility could not be determined, skipping delta calculation.")
        return None

    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * implied_volatility ** 2) * T) / (implied_volatility * np.sqrt(T))
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    return delta

# Step 5: Building a Function to Close a Position

In [ ]:
# Exit the spread by liquidating the position
def close_spread(short_symbol, long_symbol):

    # Close the long put by selling it
    trade_client.close_position(
        symbol_or_asset_id = long_symbol,
        close_options = ClosePositionRequest(qty = "1")
    )

    # Close the short put by buying it back
    trade_client.close_position(
        symbol_or_asset_id = short_symbol,
        close_options = ClosePositionRequest(qty = "1")
    )

# Step 6: Building a Function to Search for an Appropriate Options Set for a 0DTE Bull Put Vertical Spread

In [ ]:
def find_short_and_long_puts(zero_dte_options, underlying_symbol, risk_free_rate, OI_THRESHOLD, short_put_delta_range, long_put_delta_range):
    """
    Identify the short put (delta ~ -0.40) and long put (delta ~ -0.20) from the options chain.
    Returns dictionaries containing details of the selected options.
    """
    underlying_price = get_underlying_price(underlying_symbol)
    short_put = None
    long_put = None

    for option_data in zero_dte_options:
        try:
            # Validate open interest and volume
            if option_data.open_interest is None or option_data.open_interest_date is None:
                continue
            if float(option_data.open_interest) <= OI_THRESHOLD:
                continue

            # Fetch the latest option quote
            option_symbol = option_data.symbol
            option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
            option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]
            print(f"Option symbol is: {option_symbol}")

            # Extract necessary data
            option_price = (option_quote.bid_price + option_quote.ask_price) / 2
            strike_price = float(option_data.strike_price)
            expiration_date_naive_dt = datetime.combine(option_data.expiration_date, time(16, 0)) # Set time to 4:00 PM
            expiry = expiration_date_naive_dt.replace(tzinfo=timezone)
            # expiry = pd.Timestamp(option_data.expiration_date).replace(hour=16, minute=0, second=0)

            # Calculate delta
            delta = calculate_delta(
                option_price=option_price,
                strike_price=strike_price,
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type='put'
            )
            print(f"Delta is: {delta}")

            # Create dictionaries for short and long puts based on delta thresholds
            if short_put_delta_range[0] <= delta <= short_put_delta_range[1]:
                short_put = {
                    'close_price': option_data.close_price,
                    'close_price_date': option_data.close_price_date,
                    'expiration_date': expiry,
                    'id': option_data.id,
                    'name': option_data.name,
                    'open_interest': option_data.open_interest,
                    'open_interest_date': option_data.open_interest_date,
                    'root_symbol': option_data.root_symbol,
                    'size': option_data.size,
                    'status': option_data.status,
                    'strike_price': strike_price,
                    'style': option_data.style,
                    'symbol': option_symbol,
                    'tradable': option_data.tradable,
                    'type': option_data.type,
                    'underlying_asset_id': option_data.underlying_asset_id,
                    'underlying_symbol': option_data.underlying_symbol,
                    'initial_delta': delta,
                    'initial_option_price': option_price,
                }
            elif long_put_delta_range[0] <= delta <= long_put_delta_range[1]:
                long_put = {
                    'close_price': option_data.close_price,
                    'close_price_date': option_data.close_price_date,
                    'expiration_date': expiry,
                    'id': option_data.id,
                    'name': option_data.name,
                    'open_interest': option_data.open_interest,
                    'open_interest_date': option_data.open_interest_date,
                    'root_symbol': option_data.root_symbol,
                    'size': option_data.size,
                    'status': option_data.status,
                    'strike_price': strike_price,
                    'style': option_data.style,
                    'symbol': option_symbol,
                    'tradable': option_data.tradable,
                    'type': option_data.type,
                    'underlying_asset_id': option_data.underlying_asset_id,
                    'underlying_symbol': option_data.underlying_symbol,
                    'initial_delta': delta,
                    'initial_option_price': option_price,
                }

            # Stop searching if both options are found
            if short_put and long_put:
                break

        except KeyError as e:
            print(f"Error processing option {option_symbol}: {e}")
            continue

    return short_put, long_put


# Step 7: Building a Function to Execute a 0DTE Bull Put Vertical Spread

In [ ]:
def trade_0DTE_options(underlying_symbol, risk_free_rate, OI_THRESHOLD, short_put_delta_range, long_put_delta_range, min_credit_percentage, delta_stop_loss_thres, target_stop_loss_percentage):
    """
    Execute a 0DTE bull put vertical spread.
    """
    # Fetch zero DTE options
    zero_dte_options = get_0DTE_options(underlying_symbol)

    if not zero_dte_options:
        print("No 0DTE options available for the underlying symbol.")
        return

    # Find the short and long put options using the helper function
    short_put, long_put = find_short_and_long_puts(zero_dte_options, underlying_symbol, risk_free_rate, OI_THRESHOLD, short_put_delta_range, long_put_delta_range)

    if not short_put or not long_put:
        print("Could not find suitable options with 40 and 20 deltas.")
        return

    # Calculate the width of the spread
    spread_width = abs(short_put['strike_price'] - long_put['strike_price'])

    if not (2 <= spread_width <= 5):
        print(f"Spread width of {spread_width} is outside the target range of $2-$5; skipping this pair.")
        return

    # Calculate premium collected and check against threshold
    credit_received = short_put['initial_option_price'] - long_put['initial_option_price']
    min_credit_threshold = spread_width * min_credit_percentage

    if credit_received <= min_credit_threshold:
        print(f"Credit received {credit_received} is below minimum threshold {min_credit_threshold}; skipping trade.")
        return

    # Calculate profit/loss metrics
    max_profit = credit_received
    max_loss = spread_width * 100 - credit_received # Since we may have an obligation to buy 100 shares if assigned
    initial_total_delta = abs(short_put['initial_delta']) - abs(long_put['initial_delta'])
    delta_stop_loss = initial_total_delta * delta_stop_loss_thres # Default: 2 times of the initial delta when we enter the market

    print(f"Placing 0DTE Bull Put Spread on {underlying_symbol}:")
    print(f"Sell 40 Delta Put: {short_put['symbol']} (Strike: {short_put['strike_price']}, Premium: {short_put['initial_option_price']}, Delta: {short_put['initial_delta']})")
    print(f"Buy 20 Delta Put: {long_put['symbol']} (Strike: {long_put['strike_price']}, Premium: {long_put['initial_option_price']}, Delta: {long_put['initial_delta']})")
    print(f"Spread Width: {spread_width}, Credit received: {credit_received}, Max Profit: {max_profit}, Max Loss: {max_loss}")
    print(f"Initial Total Delta: {initial_total_delta}, Delta Stop-Loss Trigger: {delta_stop_loss}")


    # Create a list for the order request
    order_legs = []
    ## Append Long Put
    order_legs.append(OptionLegRequest(
        symbol=long_put["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))
    ## Append Short Put
    order_legs.append(OptionLegRequest(
        symbol=short_put["symbol"],
        side=OrderSide.SELL,
        ratio_qty=1
    ))
    # Place the order for both legs simultaneously
    req = MarketOrderRequest(
        qty=1,
        order_class=OrderClass.MLEG,
        time_in_force=TimeInForce.DAY,
        legs=order_legs
    )
    trade_client.submit_order(req)
    print("Spread order placed successfully.")

    # Set target profit and stop-loss levels
    target_profit_price = credit_received * target_stop_loss_percentage  # Default: 50% of credit received

    # Monitor the spread for delta stop-loss and profit target
    while True:
        # Fetch the latest quote for each leg
        short_quote_req = OptionLatestQuoteRequest(symbol_or_symbols=[short_put['symbol']])
        long_quote_req = OptionLatestQuoteRequest(symbol_or_symbols=[long_put['symbol']])

        short_quote = option_historical_data_client.get_option_latest_quote(short_quote_req)[short_put['symbol']]
        long_quote = option_historical_data_client.get_option_latest_quote(long_quote_req)[long_put['symbol']]

        # Calculate the mid-price for each option as the current option price
        short_put_price = (short_quote.bid_price + short_quote.ask_price) / 2
        long_put_price = (long_quote.bid_price + long_quote.ask_price) / 2

        # Calculate the latest delta values for each option
        current_short_delta = calculate_delta(
            option_price=short_put_price,
            strike_price=short_put['strike_price'],
            expiry=short_put['expiration_date'],
            underlying_price=get_underlying_price(underlying_symbol),
            risk_free_rate=risk_free_rate,
            option_type='put'
        )

        expiration_date_naive_dt=datetime.combine(long_put['expiration_date'], time(16, 0)) # Set time to 4:00 PM

        current_long_delta = calculate_delta(
            option_price=long_put_price,
            strike_price=long_put['strike_price'],
            expiry=short_put['expiration_date'],
            underlying_price=get_underlying_price(underlying_symbol),
            risk_free_rate=risk_free_rate,
            option_type='put'
        )

        # Calculate current total delta and spread price
        current_total_delta = abs(current_short_delta) - abs(current_long_delta)
        current_spread_price = short_put_price - long_put_price

        # Exit for target profit
        if current_spread_price <= target_profit_price:
            close_spread(short_put['symbol'], long_put['symbol'])
            print(f"Exited spread for target profit at price {current_spread_price}")
            break

        # Check if the stop-loss condition based on delta is met
        if current_total_delta >= delta_stop_loss:
            close_spread(short_put['symbol'], long_put['symbol'])
            print(f"Exited spread for delta stop-loss at total delta {current_total_delta}")
            break

        time.sleep(180)  # Check every 3 minutes


# Step 8: Executing the Strategy

In [ ]:
# Main block to place and monitor a single 0DTE trade
if __name__ == '__main__':
    try:
        trade_0DTE_options(underlying_symbol, risk_free_rate, OI_THRESHOLD, SHORT_PUT_DELTA_RANGE, LONG_PUT_DELTA_RANGE, MIN_CREDIT_PERCENTAGE, DELTA_STOP_LOSS_THRES, TARGET_STOP_LOSS_PERCENTAGE) # Enter the trade once and monitor it until exit
    except Exception as e:
        print(f"Error: {e}")